In [1]:
#imports

import cv2
from cv2 import threshold
import numpy as np
import tracker.tracker as tracker
import identifier.identifier as identifier
%load_ext autoreload
%autoreload 2

In [13]:
# initial setup

threshold = 20
webcam = cv2.VideoCapture(0)
ret, frame = webcam.read()

# Will's webcam dims are: 480, 640, 3
height, width, d = frame.shape

print(height, width)

#this should turn drawing area into square
line_x = width - height
box_width = 50 #arbitrarily set

#continas all plotted points and their color
# (x, y, color)
points = []
current_color = 0
colors = [(255, 0, 0, 0), (0, 255, 0, 0), (0, 0, 255, 0)]
just_changed = False

# this is used to creat the test data
# set to false when not recording webcam
recording = True
if recording:
    out = cv2.VideoWriter("working.avi", cv2.VideoWriter_fourcc(*'MJPG'), 10,(width, height)) 

xcenter_bw = 240
ycenter_bw = 240

xcenter = 400
ycenter = 240

sync_finger = None
sync_finger_bw = None
sync_frame = None
next_frame = None
while(True):
		
    # Capture the video frame
    # by frame
    if next_frame is None:
        ret, frame = webcam.read()
        _, next_frame = webcam.read()
    else:
        frame = next_frame
        _, next_frame = webcam.read()


    # flip so it makes sense to the user for drawing
    frame = cv2.flip(frame, 1)

    # duplicate current frame
    # one is used for processing the other is show to user
    c_frame = frame
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # sync with background by pressing 's'
    if cv2.waitKey(1) & 0xFF == ord('s'):
        sync_frame = frame[:, line_x:]

    # clear colors
    if cv2.waitKey(1) & 0xFF == ord('c'):
        points = []




    if sync_frame is not None:

        #  remove background from current frame and make bw
        cut_frame =  frame[:, line_x:]
        cut_frame = cv2.absdiff(sync_frame, cut_frame)
        cut_frame = cv2.blur(cut_frame, (15,15), cv2.BORDER_DEFAULT) 
        im_bw = cv2.threshold(cut_frame, threshold, 255, cv2.THRESH_BINARY)[1]


        coords = identifier.get_fingertip(im_bw)

        fingers = len(coords)

        for coord in coords:
            x, y = coord

            cv2.circle(im_bw, (x, y), 10, (255, 255, 255), 2)

            cv2.circle(c_frame, (x + line_x, y), 10, (255, 0, 0), 2)


        if fingers == 1:
            points.append((x + line_x, y, colors[current_color]))
            just_changed = False

        if fingers == 3 and not just_changed:
            current_color += 1
            just_changed = True
            if current_color > 2:
                current_color = 0



        cv2.imshow('bin', im_bw)

    if cv2.waitKey(1) & 0xFF == ord('i'):
        cv2.imwrite("color.png", c_frame)
        cv2.imwrite("bw.png", im_bw)



    # Display the resulting frame
    # this will be what the user ends up seeing
    cv2.line(c_frame, (line_x, -10), (line_x, height + 10), (0, 255, 0), thickness=3)
    


    #draw points on canvas
    for point in points:
        cv2.circle(c_frame, (point[0], point[1]), 10, point[2], -1)

    if recording and sync_frame is not None:
        out.write(c_frame)

    cv2.imshow('frame', c_frame)
    




# After the loop release the cap object
webcam.release()
if recording:
    out.release()

# Destroy all the windows
cv2.destroyAllWindows()



480 640
